In [1]:
# stock market performance while the government is divided
# divided government is a year where the house, senate, 
# or presidency is controlled by a different branch

# does the stock market do better when democrats or republicans control the house?

# does the stock market do better when democrats or republicans control the senate?

# does the stock market do better when democrats or republicans control the presidency?

# does the stock market do better when there's divided government?

# divided government with a republican president?

# divided government with a democratic president?

In [3]:
import pandas as pd

In [18]:
# party_divisions
# source: https://en.wikipedia.org/wiki/Divided_government_in_the_United_States

df_party_divisions = pd.read_fwf(
    'raw_party_divisions.txt'
    #,delimiter='\t'
    ,names=['Year', 'President', 'Senate', 'House']
    ,widths=[9,2,3,2]
    ,encoding='utf-8'
    ,skiprows=1
)
df_party_divisions.head()

,Year,President,Senate,House
0,1901–1903,R,R,R
1,1903–1905,R,R,R
2,1905–1907,R,R,R
3,1907–1909,R,R,R
4,1909–1911,R,R,R


In [24]:
def start_year(row):
    return int(row['Year'][:4])

df_party_divisions['start_year'] = df_party_divisions.apply(start_year, axis=1)
df_party_divisions.head()

,Year,President,Senate,House,start_year
0,1901–1903,R,R,R,1901
1,1903–1905,R,R,R,1903
2,1905–1907,R,R,R,1905
3,1907–1909,R,R,R,1907
4,1909–1911,R,R,R,1909


In [30]:
df_party_divisions_annual = pd.DataFrame(columns=['Year', 'President', 'Senate', 'House'])

for row in df_party_divisions.iterrows():
    row_dict = {
        'Year': row[1][4]
        ,'President': row[1][1]
        ,'Senate': row[1][2]
        ,'House': row[1][3]
    }
    
    df_party_divisions_annual = df_party_divisions_annual.append(row_dict, ignore_index=True)
    
    row_dict['Year'] += 1
    df_party_divisions_annual = df_party_divisions_annual.append(row_dict, ignore_index=True)
    
df_party_divisions_annual.head()

,Year,President,Senate,House
0,1901,R,R,R
1,1902,R,R,R
2,1903,R,R,R
3,1904,R,R,R
4,1905,R,R,R


In [36]:
def divided_government(row):
    if (row['President'] == 'R' and row['Senate'] == 'R' and row['House'] == 'R') or (row['President'] == 'D' and row['Senate'] == 'D' and row['House'] == 'D'):
        return False
    else:
        return True
    
df_party_divisions_annual['Divided Government'] = df_party_divisions_annual.apply(divided_government, axis=1)
df_party_divisions_annual.head()

,Year,President,Senate,House,Divided Government
0,1901,R,R,R,False
1,1902,R,R,R,False
2,1903,R,R,R,False
3,1904,R,R,R,False
4,1905,R,R,R,False


In [78]:
# sp500_returns
# source: http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html
df_sp500 = pd.read_fwf(
    'raw_sp500_returns.txt'
)

df_sp500_clean = pd.DataFrame(columns=['Year', 'S&P500 Return'])

for row in df_sp500.iterrows():
    row_dict = {
        'Year': row[1][0]
        ,'S&P500 Return': row[1][1][:6]
    }
    
    row_dict['S&P500 Return'] = row_dict['S&P500 Return'].replace('%', '')
    row_dict['S&P500 Return'] = row_dict['S&P500 Return'].replace('\t', '')

    df_sp500_clean = df_sp500_clean.append(row_dict, ignore_index=True)
    
df_sp500_clean.head()

,Year,S&P500 Return
0,1928,43.81
1,1929,-8.30
2,1930,-25.12
3,1931,-43.84
4,1932,-8.64


In [81]:
df_main = pd.merge(df_party_divisions_annual, df_sp500_clean, on='Year')
df_main.to_csv('divided_government_dataset.csv')
df_main.head()

,Year,President,Senate,House,Divided Government,S&P500 Return
0,1928,R,R,R,False,43.81
1,1929,R,R,R,False,-8.30
2,1930,R,R,R,False,-25.12
3,1931,R,R,D,True,-43.84
4,1932,R,R,D,True,-8.64
